This project has the aim of forecasting sales numbers with the help of machine learning algorithms. It has several variations like daily foreast, daily forecast with zip code details and monthly forecast.

Normally, classical statistical approaches are used to forecast time series data. But there are several approaches to use machine learning and deep learning algorithms for this purpose. In this project, I tried to use both of them in my experiments.

First task, and detail of this Jupyter Notebook, is to preprocess the raw data into a suitable ML format. The raw data I acquired was in the tabular form of rows as combination and columns as historical months. For a 4 year timespan, it had about 48 columns.

Initial approach was to transform this input data into a moving months format. Afterwards, adding additional input variables that may be useful for predicting the sales of target month.

First of all, raw data format must be in a standard form as code is working in a static way to transform it. There must be the data related to combination of from-to countries and then historical sales date in columnwise order. Months to be forecasted must be included at the end of the table.

Code transforms this columnwise format to a rowwise format. Parametricaly, you choose "how many past months do you want to include as input columns".

For example, if you want to predict month N and if you choose this parameter as X, you will include all months between N-X/N-1 as input variables to predict month N sales. It is the classical approach of forecasting actually.

If we have 48 months of historical data; for each combination we will have about 48 rows of input data with this format, each row corresponds to a different month with past X months as inputs to predict that specific month, in a moving pattern.

In [14]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import more_itertools as mi
import re

pd.set_option('display.max_columns',25)

In [15]:
test = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\12M\Müşteri kırılımlı - 2015-2019 Değerleri.xlsx', sheet_name = 'Gerçekleşen2019')

test.replace(np.nan, 0, inplace = True)

#various parameters that defined
predmonths = 12 #how many months to predict
param = 12 # how many months will be taken as input variable; it can be further filtered at modelling step, so 12 will be fine here
diff = 3 #last X month's change ratios will be prepared as input
subsets = 1 #it is 1 if a segmented model will be constructed; for final version it is 1

In [16]:
#rewriting months to be predicted as 0
test.iloc[:,-(predmonths+1):] = 0

#sums a specific value rowwise, then divides by total nb of available columns
test['zeros'] = ((test == 0).astype(int).sum(axis=1) - (predmonths+1)) / (len(test.columns) - (predmonths+1+2))

#Total sales for last 4 months
test['lastmonths'] = test.iloc[:,-(predmonths+4+2)] + test.iloc[:,-(predmonths+3+2)] + test.iloc[:,-(predmonths+2+2)] + test.iloc[:,-(predmonths+1+2)]

# If there is no sales at last 4 months; or 0 percentage is above a threshold; ignores this data
ignored = test[(test['lastmonths'] == 0) | (test['zeros'] > 0.95)]
test = test[(test['lastmonths'] != 0) & (test['zeros'] <= 0.95)]

In [17]:
#construction of segments according to last 3 month sale volumes
if subsets == 1:

    conditions = [((test['lastmonths'] - test.iloc[:,-(predmonths+4+2+1)])/3 < 10),
                  ((test['lastmonths'] - test.iloc[:,-(predmonths+4+2+1)])/3 >= 10) & ((test['lastmonths'] - test.iloc[:,-(predmonths+4+2+1)])/3 < 100),
                  ((test['lastmonths'] - test.iloc[:,-(predmonths+4+2+1)])/3 >= 100)]
    choices = [3,2,1]
    test['segment'] = np.select(conditions, choices, default=3)

In [18]:
#Some of the columns that wont be used
test.drop(['lastmonths','zeros'], axis = 1, inplace = True)
ignored.drop(['lastmonths','zeros'], axis=1, inplace = True)

#variables that define column and row numbers
colnum = len(test.columns)
rownum = len(test)

#list of all columns
allcols = list(test.columns.values)

In [19]:
header_dict = {'OCK ':'01-','ŞBT ':'02-','MAR ':'03-', 'NİS ':'04-', 'MAY ':'05-', 
               'HAZ ':'06-', 'TEM ':'07-', 'AGU ':'08-', 'EYL ':'09-', 'EKM ':'10-', 'KSM ':'11-', 'ARL ':'12-'}

#Changes the names of month columns
for i in range(4,len(allcols)):
    for key in header_dict.keys():
        if (key in allcols[i]) == True:
            allcols[i] = allcols[i].replace(allcols[i][0:4], header_dict[key])

# we set the first fixed cols in this list. Then append several others according to param, to the column list
cols = ['code', 'service', 'from', 'to', 'date']
for p in range(max(predmonths,max(param,12))+1):
    if p < max(predmonths,max(param,12)):
        cols.append('M'+'-'+str(max(predmonths,max(param,12))-p)) 
    else:
        cols.append('M')
        cols.append('segment')
print(cols)

['code', 'service', 'from', 'to', 'date', 'M-12', 'M-11', 'M-10', 'M-9', 'M-8', 'M-7', 'M-6', 'M-5', 'M-4', 'M-3', 'M-2', 'M-1', 'M', 'segment']


Above code sets the column names to be used after transformation of raw data

Below code creates lists from raw rowwise data in a moving pattern according to LastN Month parameter. Firstly it sets static columns for that row in each list; then it sets the date parameter then it gets Last N months + day to be predicted.

After it completed all operations on raw data rows, dateset is turned into a Dataframe from list of lists.

In [20]:
mid = []
outlist = []

for i in range(rownum):
    rowcnt = 0
    colcnt = 0
    shift = 0
    for j in range(colnum - max(predmonths,max(param,12)) - 4 - subsets):
        while colcnt < 4:
            mid.append(test.iloc[i,colcnt])
            colcnt = colcnt + 1
        if colcnt == 4:
            mid.append(allcols[colcnt+j+max(predmonths,max(param,12))])
            colcnt = colcnt + 1
        for m in range(4+shift,4+max(predmonths,max(param,12))+shift+1):
                if m < 4 + max(predmonths,max(param,12)) + shift:
                    mid.append(test.iloc[i,m])
                else:
                    mid.append(test.iloc[i,m]) 
        shift = shift + 1
        rowcnt = rowcnt + 1
        colcnt = 0
        mid.append(test['segment'].iloc[i])
        outlist.append(mid)
        mid = []

# Create a datafram from list of lists above
out = pd.DataFrame(outlist, columns = cols)

print(out.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68257 entries, 0 to 68256
Data columns (total 19 columns):
code       68257 non-null object
service    68257 non-null object
from       68257 non-null object
to         68257 non-null object
date       68257 non-null object
M-12       68257 non-null float64
M-11       68257 non-null float64
M-10       68257 non-null float64
M-9        68257 non-null float64
M-8        68257 non-null float64
M-7        68257 non-null float64
M-6        68257 non-null float64
M-5        68257 non-null float64
M-4        68257 non-null float64
M-3        68257 non-null float64
M-2        68257 non-null float64
M-1        68257 non-null float64
M          68257 non-null float64
segment    68257 non-null int64
dtypes: float64(13), int64(1), object(5)
memory usage: 9.9+ MB
None


In [21]:
#Constructing date related variables
out['date'] = pd.to_datetime(out['date'], format = '%m-%Y') 
out['date'] = out['date'].dt.to_period('m') 

out['month'] = out['date'].dt.month.astype(str)
out['quarter'] = out['date'].dt.quarter.astype(str)
out['year'] = out['date'].dt.year.astype(str)

In [22]:
#Part that will transform country names into country codes inside input table
cntdict = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\12M\Müşteri kırılımlı - 2015-2019 Değerleri.xlsx', sheet_name = 'CountryCode')
cntdict = dict(zip(cntdict['CNT'], cntdict['CODE'])) 

#change a column with help of dictionary keys, return values
out['from'] = out['from'].map(cntdict) 
out['to'] = out['to'].map(cntdict)

#gets unique values as a list from 2 different columns
cntlist = np.unique(out[['from', 'to']].values.ravel('K')) 

#counts nonzero months per combination
out['nonzeros'] = out.groupby(['code','from','to'])['M'].transform(np.count_nonzero)

#gets the mean of nonzero combinations per quarter, month to find month and quarter indexes at next step
out['quarteravg'] = out[out['nonzeros']>=29].groupby(['code', 'from', 'to','quarter'])['M'].transform(np.mean)
out['monthavg'] = out[out['nonzeros']>=29].groupby(['code', 'from', 'to','month'])['M'].transform(np.mean)
out['allavg'] = out[out['nonzeros']>=29].groupby(['code', 'from', 'to'])['M'].transform(np.mean)

#for rare events indexes are calculated as 1, otherwise it is calculated with formula
out['quarterind'] = np.where(out['nonzeros'] < 29, 1,out['quarteravg']/out['allavg'])

out['monthind'] = np.where(out['nonzeros'] < 29, 1, out['monthavg']/out['allavg'])

In [23]:
# code to calculate moving averages
movinglist = []

for i in [3,6,12]:
    movinglist = []
    for col in out.columns:
    
        if ('M-' in col):
        
            if (int(re.findall('\d+', col)[0])) <= i:
                movinglist.append(col)
    out['moving'+str(i)] = (out[movinglist].sum(axis=1))/i

out.drop(columns = ['quarteravg', 'monthavg', 'allavg', 'nonzeros'], inplace = True)

out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68257 entries, 0 to 68256
Data columns (total 27 columns):
code          68257 non-null object
service       68257 non-null object
from          68257 non-null object
to            68257 non-null object
date          68257 non-null period[M]
M-12          68257 non-null float64
M-11          68257 non-null float64
M-10          68257 non-null float64
M-9           68257 non-null float64
M-8           68257 non-null float64
M-7           68257 non-null float64
M-6           68257 non-null float64
M-5           68257 non-null float64
M-4           68257 non-null float64
M-3           68257 non-null float64
M-2           68257 non-null float64
M-1           68257 non-null float64
M             68257 non-null float64
segment       68257 non-null int64
month         68257 non-null object
quarter       68257 non-null object
year          68257 non-null object
quarterind    68257 non-null float64
monthind      68257 non-null float64
moving3   

In [24]:
# code part to append exchange rates variables from another historical table
pd.set_option('display.max_columns',25)

kur = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\Gerçekleşen.xlsx', sheet_name = 'Kurlar')

kur.drop(['Gün'], axis = 1, inplace =True)

#forward fill for specific columns and all rows; to fill empty day values with previous days
kur.loc[:,['Dolar', 'Euro']] = kur.loc[:,['Dolar', 'Euro']].ffill(axis = 0)
kur.loc[:,['Dolar', 'Euro']] = kur.loc[:,['Dolar', 'Euro']].bfill(axis = 0)

#getting time values from datetime object
kur['month'] = kur['Tarih'].dt.month.astype(str)
kur['quarter'] = kur['Tarih'].dt.quarter.astype(str)
kur['year'] = kur['Tarih'].dt.year.astype(str)

kur['Tarih'] = kur['Tarih'].dt.strftime('%m-%Y') #from datetime to string

#kur['Dolarx'] = kur.groupby(pd.TimeGrouper(freq = 'D'))['Dolar'].mean()
#kur['Dolarx'] = kur.groupby(['month', 'year'], as_index = False)['Dolar'].mean()
#kur['Dolar'] = kur.groupby(['Tarih'])['Dolar'].mean()
#normally groupby changes the structure of dataframe and output is summary table, only grouped columns and aggregation
#but transfrom changes it, uses the same data structure. !!!!

# to get monthly average exchange rates
kur['Dolar'] = kur.groupby('Tarih')['Dolar'].transform(np.mean)
kur['Euro'] = kur.groupby('Tarih')['Euro'].transform(np.mean)


kur.drop_duplicates(inplace=True)
kur.reset_index(inplace=True, drop=True)

kur.info()

#joining two tables, selecting only specific columns from right table, join on multiple columns.
final = pd.merge(out, kur[['Dolar', 'Euro', 'month', 'year']], how = 'left', on = ['month', 'year'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1367 entries, 0 to 1366
Data columns (total 7 columns):
Tarih      1367 non-null object
Dolar      1367 non-null float64
Euro       1367 non-null float64
Pound      1307 non-null float64
month      1367 non-null object
quarter    1367 non-null object
year       1367 non-null object
dtypes: float64(3), object(4)
memory usage: 74.9+ KB


In [25]:
pd.set_option('display.max_columns',25)

holiday = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\tatiller.xlsx', sheet_name = 'Tatiller')

holiday['HOLIDAY'] = pd.to_datetime(holiday['HOLIDAY'], format = '%d/%m/%Y')

#get date related variables from da datetime variable
holiday['day'] = holiday['HOLIDAY'].dt.day.astype(str)
holiday['weekday'] = holiday['HOLIDAY'].dt.weekday
holiday['month'] = holiday['HOLIDAY'].dt.month.astype(str)
holiday['year'] = holiday['HOLIDAY'].dt.year.astype(str)

#case when structure basically, if holiday value is x then set 1 else 0
holiday['ismon1'] = np.where(holiday['weekday'] == 1, 1,0)
holiday['istue1'] = np.where(holiday['weekday'] == 2, 1,0)
holiday['iswed1'] = np.where(holiday['weekday'] == 3, 1,0)
holiday['isthurs1'] = np.where(holiday['weekday'] == 4, 1,0)
holiday['isfriday1'] = np.where(holiday['weekday'] == 5, 1,0)


#I wanted to count FRİDAYS according to partition. i found a way by flagging fridays above. Then sum these flags over partition. Transform was required to hold original table
holiday['moncnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['ismon1'].transform(np.sum)
holiday['tuecnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['istue1'].transform(np.sum)
holiday['wedcnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['iswed1'].transform(np.sum)
holiday['thurscnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['isthurs1'].transform(np.sum)
holiday['fridaycnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['isfriday1'].transform(np.sum)

#holiday['cnt'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'], as_index = False).size().reset_index(name='cnt')['cnt']
#groupby ile yapılan aggregation işlemleri özet tabloyu getiriyor. as_index yapınca değerlerin index olmasını engelliyorsun. size fonksiyonu count alıyor, bu grupta kaç satır var gibi.
#reset index de aldığın count için sütun açmanı sağlıyor. gelen sonuç groupby içindeki 3 sütunu ve sonuç sütununu içeriyor, yani 4 sütunlu bir dataframe
#istediğimiz tek bir sütun olduğu için de o dataframe içinden ilgilendiğiöiz sütunu çekerek eşliyoruz. Ama transform gerektiği için bu yapıyı kullanmadık

#holiday table shows holidays rowwise. This line counts total holidays per month
holiday['holidaycnt'] = holiday.groupby(['COUNTRY_CODE', 'month','year'])['COUNTRY_CODE'].transform('count')

#transfrom ile yapınca groupbyda yapılan özetleme işlemine girmiyor, tüm satırlara duplike de olsa aggregate değerlerini getiriyor. 

In [26]:
#This creates a full schedule to calculate fridays for each month historically
dat = pd.date_range(start = '1/1/2015', end='12/12/2020')

dat = pd.DataFrame(dat, columns = ['date'])

dat['month'] = dat['date'].dt.month.astype(str)
dat['year'] = dat['date'].dt.year.astype(str)
dat['weekday'] = dat['date'].dt.weekday

dat['ismon2'] = np.where(dat['weekday'] ==1, 1,0)
dat['istue2'] = np.where(dat['weekday'] ==2, 1,0)
dat['iswed2'] = np.where(dat['weekday'] ==3, 1,0)
dat['isthurs2'] = np.where(dat['weekday'] ==4, 1,0)
dat['isfriday2'] = np.where(dat['weekday'] ==5, 1,0)


dat['moncnt2'] = dat.groupby(['month', 'year'], as_index = False)['ismon2'].transform(np.sum)
dat['tuecnt2'] = dat.groupby(['month', 'year'], as_index = False)['istue2'].transform(np.sum)
dat['wedcnt2'] = dat.groupby(['month', 'year'], as_index = False)['iswed2'].transform(np.sum)
dat['thurscnt2'] = dat.groupby(['month', 'year'], as_index = False)['isthurs2'].transform(np.sum)
dat['fridaycnt2'] = dat.groupby(['month', 'year'], as_index = False)['isfriday2'].transform(np.sum)

dat.drop(columns=['date', 'weekday','isfriday2', 'isthurs2', 'iswed2', 'istue2', 'ismon2'], inplace = True)
dat.drop_duplicates(inplace=True)
dat.reset_index(inplace=True, drop=True)

In [27]:
#this part cross joins country list with monthly friday table. Creates full schedule for each country in list
countries = cntlist
cnt = pd.DataFrame(countries, columns = ['COUNTRY_CODE'])

#there is not a common column for both tables. We created a fictional column to join
dat['key'] = 1
cnt['key'] = 1

all = pd.merge(cnt, dat, how = 'outer', on='key')

all.drop(columns = ['key'], inplace = True)
dat.drop(columns = ['key'], inplace = True)
cnt.drop(columns = ['key'], inplace = True)

#this part merges all countries and monthly fridays table with holiday table via left join, so that we can see them integrated. 
all = pd.merge(all, holiday[['COUNTRY_CODE', 'month', 'year','moncnt1', 'tuecnt1', 'wedcnt1', 'thurscnt1', 'fridaycnt1', 'holidaycnt']], 
               how = 'left', left_on = ['COUNTRY_CODE', 'month', 'year'], right_on = ['COUNTRY_CODE', 'month', 'year'])

all.drop_duplicates(inplace=True)
all.replace(np.nan, 0, inplace = True)

all['moncnt'] = all['moncnt2'] - all['moncnt1']
all['tuecnt'] = all['tuecnt2'] - all['tuecnt1']
all['wedcnt'] = all['wedcnt2'] - all['wedcnt1']
all['thurscnt'] = all['thurscnt2'] - all['thurscnt1']
all['fridaycnt'] = all['fridaycnt2'] - all['fridaycnt1'] #count of work fridays, some fridays may intersect with holiday fridays

all.drop(columns = ['fridaycnt2', 'fridaycnt1', 'thurscnt2', 'thurscnt1', 'wedcnt2', 'wedcnt1', 'tuecnt2', 'tuecnt1', 'moncnt2', 'moncnt1'], inplace = True)

In [28]:

final = pd.merge(final, all, how = 'left', left_on = ['from', 'month', 'year'], right_on = ['COUNTRY_CODE', 'month','year'])

final.drop(columns = ['COUNTRY_CODE'], inplace = True)

final.rename(columns = {'holidaycnt':'dptholidaycnt', 'fridaycnt': 'dptfridaycnt', 'thurscnt' : 'dptthurscnt', 'wedcnt': 'dptwedcnt', 
                        'tuecnt': 'dpttuecnt', 'moncnt':'dptmoncnt'}, inplace = True)

final = pd.merge(final, all, how = 'left', left_on = ['to', 'month', 'year'], right_on = ['COUNTRY_CODE', 'month', 'year'])

final.drop(columns = ['COUNTRY_CODE'], inplace = True)

final.rename(columns = {'holidaycnt':'arvholidaycnt', 'fridaycnt':'arvfridaycnt', 'thurscnt': 'arvthurscnt', 'wedcnt': 'arvwedcnt',
                        'tuecnt':'arvtuecnt', 'moncnt':'arvmoncnt'}, inplace = True)

#print_excel = final.to_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\PrepForecastPWCompany10.xlsx', index = None, header = True, sheet_name = 'FullTable')
#print_csv = final.to_csv(r'C:\Users\ali.kilinc\Desktop\Tahminleme\PrepForecastPWCompany10.csv', index = None, header = True)
